# Case Study - Berpikir Komputasional

```plaintext
Nama: Edgrant Henderson Suryajaya
NPM: 2206025016
```

# Case Study 6: Autonomous Drone Delivery System

## Background:

With the rise of e-commerce, companies are exploring autonomous drone delivery systems to improve delivery speed and reduce costs. However, operating drones in urban or rural environments presents challenges such as avoiding obstacles, adapting to weather conditions, and managing battery life.

## Problem Statement:

Design a computational solution to optimize drone delivery routes. The solution should consider:

1. Real-time data (e.g., weather conditions, air traffic, and battery levels).
2. Obstacle avoidance and safe navigation.
3. Delivery priorities (e.g., urgent packages).
4. Energy efficiency and battery life optimization.

Your goal is to create a system that dynamically plans and adjusts delivery routes for autonomous drones based on real-time data while balancing delivery speed, energy efficiency, and safety. Additionally, your solution must incorporate at least one classic algorithm we have learned, such as linear search, finding extreme values, greedy algorithm, or brute force.

In [397]:
# Util Class declaration

class Sight:
    def __init__(self, left_object, center_object, right_object):
        self.left_object = left_object
        self.center_object = center_object
        self.right_object = right_object
    
    def get_left_object(self):
        return self.left_object
    
    def get_center_object(self):
        return self.center_object
    
    def get_right_object(self):
        return self.right_object
    
    def set_left_object(self, left_object):
        self.left_object = left_object
        
    def set_center_object(self, center_object):
        self.center_object = center_object
        
    def set_right_object(self, right_object):
        self.right_object = right_object

    def clear_sight(self):
        self.left_object = ""
        self.center_object = ""
        self.right_object = ""

class Weather:
    def __init__(self, wind_speed, rain, visibility):
        self.wind_speed = wind_speed
        self.rain = rain
        self.visibility = visibility

    def get_wind_speed(self):
        return self.wind_speed

    def get_rain(self):
        return self.rain
    
    def get_visibility(self):
        return self.visibility
    
class Location:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude
    
    def get_latitude(self):
        return self.latitude
    
    def get_longitude(self):
        return self.longitude
    
    def set_latitude(self, latitude):
        self.latitude = latitude
        
    def set_longitude(self, longitude):
        self.longitude = longitude

class Route:
    def __init__(self, normal_location: list[Location]):
        self.normal_location = normal_location

    def get_location(self, index):
        if self.normal_location:
            return self.normal_location[index]
        else:
            return None

    def set_route(self, normal_location):
        self.normal_location = normal_location

    def enqueue_location(self, location):
        self.normal_location.append(location)

    def dequeue_location(self):
        if self.normal_location:
            return self.normal_location.pop(0)
        else:
            return None
        
class Power_Metrics:
    def __init__(self, wind_speed_coefficient : float = 0.1, rain_coefficient : float = 0.2, visibility_coefficient : float = 0.1, speed_coefficient : float = 0.1):
        self.wind_speed_coefficient = wind_speed_coefficient
        self.rain_coefficient = rain_coefficient
        self.visibility_coefficient = visibility_coefficient
        self.speed_coefficient = speed_coefficient

In [398]:
class Drone:
    def __init__(self, name, max_speed : float, max_battery: int, route: Route, number_of_priority : int, weather: Weather, sight: Sight = None, starting_speed: float = 20, power_metrics: Power_Metrics = Power_Metrics()):
        self.name = name
        self.max_speed = max_speed
        self.route = route
        self.number_of_priority = number_of_priority
        self.battery = float(max_battery) # Ensure battery is float
        self.speed = starting_speed # in m/s
        self.starting_speed = starting_speed
        self.sight = sight
        self.weather = weather
        self.power_metrics = power_metrics
        self.delivered_node = 0

    def calculate_distance(self):
        if len(self.route.normal_location) < 2:
            # Not enough points to define a travel segment from current to next.
            return 0.0
        
        loc1 = self.route.get_location(0) # Current location
        loc2 = self.route.get_location(1) # Next location
        
        # print(f"Location 1 (current): {loc1.get_latitude()}, {loc1.get_longitude()}")
        # print(f"Location 2 (next): {loc2.get_latitude()}, {loc2.get_longitude()}")

        lat1 = loc1.get_latitude()
        lon1 = loc1.get_longitude()
        lat2 = loc2.get_latitude()
        lon2 = loc2.get_longitude()

        distance = ((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2) ** 0.5
        return distance

    def calculate_duration(self):
        if len(self.route.normal_location) < 2: # No travel segment
            return 0.0
        
        distance = self.calculate_distance() # Distance from current (route[0]) to next (route[1])
        if self.speed > 0:
            duration = distance / self.speed
            return duration
        return 0.0
        
    def calculate_power_consumption(self):
        if len(self.route.normal_location) < 2: # No travel segment
            return 0.0

        power_consumption = 0.0

        wind_speed_power = self.weather.get_wind_speed() * self.power_metrics.wind_speed_coefficient
        rain_power = self.weather.get_rain() * self.power_metrics.rain_coefficient
        vissibility_power = (1000 - self.weather.get_visibility()) * self.power_metrics.visibility_coefficient
        speed_power = self.speed * self.power_metrics.speed_coefficient

        distance = self.calculate_distance() # For travel from route[0] to route[1]
        distance_power = distance * 0.5

        power_consumption = wind_speed_power + rain_power + vissibility_power + speed_power + distance_power
        
        # The detailed print from here was moved to move_to_location for better context
        
        return power_consumption

    def add_obstacle(self, sight: Sight):
        if isinstance(sight, Sight):
            self.sight = sight
            print(f"Obstacle detection system set with left: {sight.get_left_object()}, center: {sight.get_center_object()}, right: {sight.get_right_object()}")
        else:
            print("Invalid sight object. Please provide a valid Sight instance.")
    
    def handle_obstacle(self):
        if self.sight:
            left_object = self.sight.get_left_object()
            center_object = self.sight.get_center_object()
            right_object = self.sight.get_right_object()

            if (left_object or center_object or right_object) and len(self.route.normal_location) > 1:
                print(f"Obstacle detected towards {self.route.normal_location[1]}! Re-routing.")
                obstructed_target = self.route.normal_location.pop(1) 
                self.route.enqueue_location(obstructed_target) 
                print(f"Re-routed. New next target is {self.route.normal_location[1] if len(self.route.normal_location) > 1 else 'end of route'}.")
                self.sight.clear_sight()
                # self.move_to_location()
                return True
            elif not (left_object or center_object or right_object):
                print("No obstacles detected.")
                return False
            else: # Obstacle detected but no way to re-route (e.g. only current location left)
                print("Obstacle detected, but no alternative immediate next location to re-route to.")
                return False
        else:
            return False
    
    def change_battery(self): # This method seems redundant if move_to_location handles battery.
        power_consumption = self.calculate_power_consumption() 
        self.battery -= power_consumption
        
        if self.battery < 0:
            self.battery = 0
            return False
        else:
            return True
    
    def last_location(self):
        if self.route.normal_location:
            return self.route.normal_location[-1]
        else:
            return None

    def move_to_location(self):
        if not self.route.normal_location:
            print(f"Drone {self.name} has no route.")
            return False 

        current_location_object = self.route.normal_location[0]

        if len(self.route.normal_location) > 1: 
            next_location_object = self.route.normal_location[1]

            if self.delivered_node < self.number_of_priority:
                print(f"Drone {self.name} is moving to priority location: {next_location_object}.")
                self.speed = self.max_speed
            else:
                print(f"Drone {self.name} is moving to normal location: {next_location_object}.")
                self.speed = self.starting_speed
                
            power_needed = self.calculate_power_consumption() 
            duration = self.calculate_duration() 

            if power_needed > 0 : # Print details if there's actual travel
                wind_speed_power = self.weather.get_wind_speed() * self.power_metrics.wind_speed_coefficient
                rain_power = self.weather.get_rain() * self.power_metrics.rain_coefficient
                vissibility_power = (1000 - self.weather.get_visibility()) * self.power_metrics.visibility_coefficient
                speed_power = self.speed * self.power_metrics.speed_coefficient
                distance_val = self.calculate_distance() # Recalculate for printing, or store from power_needed context
                distance_power_val = distance_val * 0.5
                print(f"""
                Attempting to move from {current_location_object} to {next_location_object}.
                Distance: {distance_val:.2f}, Duration: {duration:.2f}s
                Power Breakdown for this segment:
                Wind Speed Power: {round(wind_speed_power, 2)}
                Rain Power: {round(rain_power, 2)}
                Visibility Power: {round(vissibility_power, 2)}
                Speed Power: {round(speed_power, 2)}
                Distance Power: {round(distance_power_val, 2)}
                Total Power Consumption for segment: {round(power_needed, 2)}
                """)

            self.battery -= power_needed
            
            departed_loc = self.route.dequeue_location() 
            print(f"Drone {self.name} moved from {departed_loc} to {self.route.normal_location[0]} in {duration:.2f} seconds.")
            # print(f"Battery level: {self.battery:.2f} mAh") # Moved to main loop for consistent check
            self.delivered_node += 1
            return True 
        
        else: 
            final_dest = self.route.dequeue_location() 
            print(f"Drone {self.name} has reached its final listed destination: {final_dest}.")
            # print(f"Battery level: {self.battery:.2f} mAh") # Moved to main loop
            return False 
    
    def print_route(self):
        if not self.route.normal_location:
            print(f"Drone {self.name} has no route planned.")
            return
        print(f"Drone {self.name} current route (next stop is index 0):")
        for i, location in enumerate(self.route.normal_location):
            print(f"  {i}: {location}")

In [399]:
import math

class Location:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude

    def get_latitude(self):
        return self.latitude

    def get_longitude(self):
        return self.longitude

    # Added for easier comparison and removal in lists/sets
    def __eq__(self, other):
        if not isinstance(other, Location):
            return NotImplemented
        return self.latitude == other.latitude and self.longitude == other.longitude

    def __hash__(self):
        return hash((self.latitude, self.longitude))

    def __repr__(self): # For better printing in examples
        return f"Location({self.latitude}, {self.longitude})"


def calculate_distance(location1: Location, location2: Location) -> float:
    lat1 = location1.get_latitude()
    lon1 = location1.get_longitude()
    lat2 = location2.get_latitude()
    lon2 = location2.get_longitude()

    # Euclidean distance formula
    distance = math.sqrt((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2)
    return distance

def find_shortest_path_to_unvisited(current_location: Location, unvisited_nodes: list[Location]) -> tuple[int, float]:
    if not unvisited_nodes:
        return -1, float('inf')

    shortest_distance = float('inf')
    shortest_path_index = -1

    # Iterate through all unvisited nodes to find the closest one to the current_location
    for i, node in enumerate(unvisited_nodes):
        distance = calculate_distance(current_location, node)
        if distance < shortest_distance:
            shortest_distance = distance
            shortest_path_index = i

    return shortest_path_index, shortest_distance

def greedy_tsp_nearest_neighbor(nodes: list[Location], priority_nodes: list[Location] = None) -> tuple[list[Location], float]:
    if not nodes and not priority_nodes:
        return [], 0.0

    if priority_nodes is None:
        priority_nodes = []

    path: list[Location] = []
    total_distance = 0.0
    current_node = None
    start_node_of_tour = None # This will be the very first node of the path

    # Use sets for efficient membership checks
    priority_nodes_set = set(priority_nodes)
    nodes_set = set(nodes)

    # --- Determine the very first node of the tour ---
    if priority_nodes:
        # If priority nodes exist, the tour MUST start with one of them.
        # For NN on priority nodes, we can just pick the first one from the input list
        # as the initial start of this priority sub-tour.
        start_node_of_tour = priority_nodes[0]
        # Remove it from the pool of priority nodes to visit
        remaining_priority_nodes_to_visit = list(priority_nodes_set - {start_node_of_tour})
    elif nodes: # If no priority nodes but regular nodes exist
        start_node_of_tour = nodes[0]
        remaining_priority_nodes_to_visit = []
    else:
        # Should be caught by the initial check, but as a safeguard
        return [], 0.0

    path.append(start_node_of_tour)
    current_node = start_node_of_tour

    # Prepare the list of non-priority nodes for the second phase
    # This list initially contains all 'nodes' that are not priority nodes.
    remaining_non_priority_nodes = [node for node in nodes if node not in priority_nodes_set]

    # Ensure the start_node_of_tour is removed from the non-priority pool if it was originally there
    if start_node_of_tour in remaining_non_priority_nodes:
        remaining_non_priority_nodes.remove(start_node_of_tour)


    # Phase 1: Apply Nearest Neighbor TSP to the remaining priority nodes
    # The `current_node` is already set from `start_node_of_tour`
    temp_current_node_for_nn = current_node # Use a temporary var to keep track of current node within phase

    while remaining_priority_nodes_to_visit:
        next_node_index, distance_to_next = find_shortest_path_to_unvisited(temp_current_node_for_nn, remaining_priority_nodes_to_visit)

        if next_node_index == -1:
            break # No more priority nodes to visit

        next_node = remaining_priority_nodes_to_visit[next_node_index]
        total_distance += distance_to_next
        path.append(next_node)
        temp_current_node_for_nn = next_node # Update current node for next iteration within priority nodes
        remaining_priority_nodes_to_visit.pop(next_node_index)

    # After visiting all priority nodes, the current_node for the next phase
    # is the last node visited in the priority sequence (if any).
    # If there were no priority nodes, temp_current_node_for_nn remains the start_node_of_tour.
    current_node = temp_current_node_for_nn


    # Phase 2: Apply Nearest Neighbor TSP to the remaining non-priority nodes
    while remaining_non_priority_nodes:
        next_node_index, distance_to_next = find_shortest_path_to_unvisited(current_node, remaining_non_priority_nodes)

        if next_node_index == -1:
            break # No more non-priority nodes to visit

        next_node = remaining_non_priority_nodes[next_node_index]
        total_distance += distance_to_next
        path.append(next_node)
        current_node = next_node
        remaining_non_priority_nodes.pop(next_node_index)

    # Complete the cycle by returning to the original start node of the tour
    # Only add the start node if the path is not empty and the last node isn't already the start node.
    if path and path[-1] != start_node_of_tour:
        distance_back_to_start = calculate_distance(current_node, start_node_of_tour)
        total_distance += distance_back_to_start
        path.append(start_node_of_tour)

    return path, total_distance

In [400]:
from random import randint

# Generate random locations
random_nodes = []
for _ in range(5):
    random_lat = randint(0, 1000)
    random_lon = randint(0, 1000)
    random_nodes.append(Location(random_lat, random_lon))

random_nodes.append(Location(0, 0))
random_nodes.append(Location(10, 10))
random_nodes.append(Location(217, 12))

priority_nodes = [Location(1, 1), Location(500, 500), Location(6, 50), Location(400, 500)]

print("Original nodes:", random_nodes)

tour_path, tour_distance = greedy_tsp_nearest_neighbor(random_nodes, priority_nodes)

print("\nGreedy TSP Tour:")
for i, loc in enumerate(tour_path):
    print(f"Location {i}: {loc.get_latitude()}, {loc.get_longitude()}")
    if i < len(tour_path) - 1:
        print(f"Distance to next location: {calculate_distance(loc, tour_path[i + 1]):.2f}")
    else:
        print(f"Distance back to start: {calculate_distance(loc, tour_path[0]):.2f}")
print(f"\nTotal Tour Distance: {tour_distance:.2f}")


Original nodes: [Location(500, 577), Location(238, 845), Location(526, 700), Location(29, 140), Location(11, 993), Location(0, 0), Location(10, 10), Location(217, 12)]

Greedy TSP Tour:
Location 0: 1, 1
Distance to next location: 49.25
Location 1: 6, 50
Distance to next location: 598.11
Location 2: 400, 500
Distance to next location: 100.00
Location 3: 500, 500
Distance to next location: 77.00
Location 4: 500, 577
Distance to next location: 125.72
Location 5: 526, 700
Distance to next location: 322.44
Location 6: 238, 845
Distance to next location: 270.99
Location 7: 11, 993
Distance to next location: 853.19
Location 8: 29, 140
Distance to next location: 131.38
Location 9: 10, 10
Distance to next location: 14.14
Location 10: 0, 0
Distance to next location: 217.33
Location 11: 217, 12
Distance to next location: 216.28
Location 12: 1, 1
Distance back to start: 0.00

Total Tour Distance: 2975.83


In [401]:
weather = Weather(wind_speed=10, rain=20, visibility=500)

for i in range(len(tour_path)):
    tour_path[i] = Location(tour_path[i].get_latitude(), tour_path[i].get_longitude())

route = Route(tour_path)

drone = Drone(name="Drone1", max_speed=50, max_battery=3000, route=route, weather=weather, number_of_priority=3, starting_speed=20)
drone.print_route()


Drone Drone1 current route (next stop is index 0):
  0: Location(1, 1)
  1: Location(6, 50)
  2: Location(400, 500)
  3: Location(500, 500)
  4: Location(500, 577)
  5: Location(526, 700)
  6: Location(238, 845)
  7: Location(11, 993)
  8: Location(29, 140)
  9: Location(10, 10)
  10: Location(0, 0)
  11: Location(217, 12)
  12: Location(1, 1)


In [402]:
success_flag = True

counter = 0

while drone.last_location() != None:
    drone.handle_obstacle()
    drone.move_to_location()

    if not drone.battery > 0:
        print("Battery depleted. Drone cannot continue.")
        success_flag = False
        break
    else:
        print(f"Drone {drone.name} is at location {drone.last_location()}")
    print('-' * 20)

    if counter % 5 == 0:
        drone.add_obstacle(Sight("Tree", "", ""))

    counter += 1

Drone Drone1 is moving to priority location: Location(6, 50).

                Attempting to move from Location(1, 1) to Location(6, 50).
                Distance: 49.25, Duration: 0.99s
                Power Breakdown for this segment:
                Wind Speed Power: 1.0
                Rain Power: 4.0
                Visibility Power: 50.0
                Speed Power: 5.0
                Distance Power: 24.63
                Total Power Consumption for segment: 84.63
                
Drone Drone1 moved from Location(1, 1) to Location(6, 50) in 0.99 seconds.
Drone Drone1 is at location Location(1, 1)
--------------------
Obstacle detection system set with left: Tree, center: , right: 
Obstacle detected towards Location(400, 500)! Re-routing.
Re-routed. New next target is Location(500, 500).
Drone Drone1 is moving to priority location: Location(500, 500).

                Attempting to move from Location(6, 50) to Location(500, 500).
                Distance: 668.23, Duration: 13.36s

In [403]:
if success_flag:
    print(f"Drone has successfully completed the route with remaining battery: {drone.battery:.2f} mAh")
else:
    print(f"Drone failed to complete the route, with {drone.delivered_node} nodes delivered")

Drone has successfully completed the route with remaining battery: 256.88 mAh
